In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import math
import numpy as np
from numpy import random
from math import pi
import matplotlib.pyplot as plt
from math import sqrt
from  sklearn import mixture
import scipy.sparse
from scipy.stats import norm
from scipy.stats import multivariate_normal
tfd = tfp.distributions
from sklearn.utils import shuffle


In [3]:
# Unnormalized Data Distribution
def pm0(x, m, s):
    mvn = tfd.MultivariateNormalFullCovariance(
    loc=m,
    covariance_matrix=s)
    k = len(m)
    return np.sqrt(np.linalg.det(s)*(2*pi)**k)*mvn.prob(x);

# Noise Distribution
def pn(x, m0, s0):
    mvn = tfd.MultivariateNormalFullCovariance(
    loc=m0,
    covariance_matrix=s0)
    
    return mvn.prob(x);

In [29]:
## This class is a helper that  will be instantiated in objects containing results from A Gradient Descent

class Gradient:

    # instance attribute
    #Receive those attributes from
    def __init__(self, cte,mu=[],sigma=[], error_mu =[],error_sigma=[], error_cte=[], ctes=[],mus=[],sigmas=[]):
        self.cte = cte
        self.mu = mu
        self.sigma = sigma
        
        self.error_mu = error_mu
        self.error_sigma = error_sigma
        self.error_cte = error_cte 
        
        self.ctes = ctes
        self.mus = mus
        self.sigmas = sigmas
        

In [5]:
def obj_f2():
    
    g1= a*Z1 + b*Z2 + mu[0]
    g2 = c*Z1 +  d*Z2 + mu[1]

    g= tf.stack([g1,g2],1)


    p_n = pn( g,m0, S) # noise (generator) density 
    p_m = pm0(g, m_data, s_data) #data density
    return tf.math.reduce_mean( tf.math.log(p_n/(cte*p_m+p_n)))

In [27]:

######## ALTERNATIVE VERSION #############

# d =2
m_data = [1,2]
s_data = [[5, 11], [11, 25]]

####Cholesky gives : [1,] [3,4]

batch_size=100
x_batches = multivariate_normal.rvs(m_data, s_data, 10,batch_size)


mu_init = [5, -3]
a,b,c,d =  tf.Variable((-5+ 1), dtype="float32"), tf.Variable(8+2, dtype="float32"), tf.Variable(5+3, dtype="float32"), tf.Variable(1+ 4 ,dtype="float32")
cte_init = 1

mu = tf.Variable(mu_init, dtype="float32")

cte = cte_init
error_mu = [] 
error_sigma = []


error_cte = [] 

mus = []
sigmas = []


ctes = []
max_iters = 700
learning_rate = 0.01
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
 
for itr in range(max_iters): 
    #print(itr)
    for x in x_batches:
        ctes.append(cte)
        Z1 = np.array(random.normal( 0, 1,batch_size)).astype("float32")
        Z2 = np.array(random.normal( 0, 1,batch_size)).astype("float32")
        
        g1= a*Z1 + b*Z2 + mu[0]
        g2 = c*Z1 +  d*Z2 + mu[1]

        A = np.array([[a,b],[c,d]])
        S = np.array(np.matmul(A , np.transpose(A))).astype("float32")
        
       

        g= np.stack([g1,g2],1)
        
       
        
        m0 = mu.numpy()
        
        p_nx = pn( x,m0, S).numpy() # noise (generator) density 
        p_mx = pm0(x, m_data, s_data).numpy() #data density

        
        p_ng = pn( g,m0, S).numpy() # noise (generator) density 
        p_mg = pm0(g, m_data, s_data).numpy() #data density

        grad_cte = 1/cte - p_mx/(cte*p_mx+p_nx) -  p_mg/(cte*p_mg+p_ng)
        grad_cte = np.sum(grad_cte)/batch_size
            
        
        cte = cte + 0.01*grad_cte
        
#         print("gradient   " , grad_cte )
#         print("cte" , cte)
        error_cte.append( (grad_cte) ) 
        if np.isnan(grad_cte):
            break
            
        opt.minimize(obj_f2, var_list=[a,b,c,d, mu])
        

In [21]:
print(cte)
print(1/np.sqrt(np.linalg.det(s_data)*(2*pi)**len(m_data)))

0.07957778655290482
0.0795774715459475


In [33]:
A = [[a.numpy(),b.numpy()],[c.numpy(),d.numpy()]]

print(A)

print(np.matmul(A, np.transpose(A)))

mu

[[0.18164639, 2.2286787], [1.2910917, 4.830433]]
[[ 5.0000043 11.000005 ]
 [11.000005  25.       ]]


<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1.0000005, 1.9999996], dtype=float32)>

In [50]:
def GANDescent(x_batches, m, s,mu_init , A_init, cte_init , opt, rate = 0.01, max_iters = 500):    
    # d =2
    m_data = m
    s_data = s

    
    a,b =  tf.Variable(A_init[0][0], dtype="float32"), tf.Variable(A_init[0][1], dtype="float32")
    c,d = tf.Variable(A_init[1][0], dtype="float32"), tf.Variable(A_init[1][1] ,dtype="float32")


    mu = tf.Variable(mu_init, dtype="float32")

    cte = cte_init
    error_mu = [] 
    error_sigma = []
    error_cte = [] 
    mus = []
    sigmas = []
    ctes = []

    batch_size =len(x_batches[0])
    #opt = tf.keras.optimizers.SGD(learning_rate=rate[1])
    
    def obj_f2():
    
        g1= a*Z1 + b*Z2 + mu[0]
        g2 = c*Z1 +  d*Z2 + mu[1]

        g= tf.stack([g1,g2],1)


        p_n = pn( g,m0, S) # noise (generator) density 
        p_m = pm0(g, m_data, s_data) #data density
        return tf.math.reduce_mean( tf.math.log(p_n/(cte*p_m+p_n)))

    for itr in range(max_iters): 
        #print(itr)
        for x in x_batches:
            ctes.append(cte)
            Z1 = np.array(random.normal( 0, 1,batch_size)).astype("float32")
            Z2 = np.array(random.normal( 0, 1,batch_size)).astype("float32")

            g1= a*Z1 + b*Z2 + mu[0]
            g2 = c*Z1 +  d*Z2 + mu[1]

            A = np.array([[a,b],[c,d]])
            S = np.array(np.matmul(A , np.transpose(A))).astype("float32")

            g= np.stack([g1,g2],1)

            m0 = mu.numpy()

            p_nx = pn( x,m0, S).numpy() # noise (generator) density 
            p_mx = pm0(x, m_data, s_data).numpy() #data density


            p_ng = pn( g,m0, S).numpy() # noise (generator) density 
            p_mg = pm0(g, m_data, s_data).numpy() #data density

            grad_cte = 1/cte - p_mx/(cte*p_mx+p_nx) -  p_mg/(cte*p_mg+p_ng)
            grad_cte = np.sum(grad_cte)/batch_size


            cte = cte + rate*grad_cte

            if np.isnan(grad_cte):
                print("WARNING : NAN values")
                break

            opt.minimize(obj_f2, var_list=[ mu])

    result = Gradient(cte,mu.numpy(),[[a.numpy(),b.numpy()],[c.numpy(),d.numpy()]], error_cte=error_cte, ctes = ctes)
    return result


In [ ]:
# d =2
m_data = [1,2]
s_data = [[5, 11], [11, 25]]

batch_size=100
x_batches = multivariate_normal.rvs(m_data, s_data, 10,batch_size)
mu_init = [9, 9]
a,b,c,d = 1, 2, 3, 4
cte_init = 1

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

grad=GANDescent(x_batches, m_data, s_data,mu_init , [[a,b],[c,d]], cte_init , opt, rate = 0.01, max_iters = 500)

print("constant estimate",grad.cte)
print("true constant value" , 1/np.sqrt(np.linalg.det(s_data)*(2*pi)**len(m_data)))
print("mu generaor ",grad.mu)
A = grad.sigma
print("A gen", A)

print(" SIGMA = AA'", np.matmul(A, np.transpose(A)))


In [ ]:
x1 = tf.Variable(10.0, dtype ="double")
x1, x2 = reset()
opt = tf.keras.optimizers.SGD(learning_rate=0.1)
for i in range(50):
	print ('y = {:.1f}, x1 = {:.1f}, x2 = {:.1f}'.format(fu(x1, x2).numpy(), x1.numpy(), x2.numpy()))
	opt.minimize(fu_minimzie, var_list=[x1])